In [1]:
import datetime
import fiona
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
from ribasim import Node
from ribasim.nodes import tabulated_rating_curve, level_boundary
from shapely.geometry import Point

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import *
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor
from peilbeheerst_model.crossings_to_ribasim import RibasimNetwork

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
# Set paths
waterschap = "Scheldestromen"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = -0.42  # default LevelBoundary level

## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_0"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed
)
processor.run()

Swapped edge direction between Node A: 186 and Node B: 302
Swapped edge direction between Node A: 302 and Node B: 92
Swapped edge direction between Node A: 190 and Node B: 381
Swapped edge direction between Node A: 381 and Node B: 29
Swapped edge direction between Node A: 29 and Node B: 227
Swapped edge direction between Node A: 227 and Node B: 33
Processed all actions
The target levels (streefpeilen) have been updated.
The function of the pumps have been updated.


#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

# Parameterization

## Nodes

### Basin (characteristics)

In [5]:
# Check basin area
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


In [6]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [7]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

In [8]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [9]:
#add an TRC and edges to the newly created level boundary
new_node_id = ribasim_param.get_current_max_nodeid(ribasim_model=ribasim_model) + 1

level_boundary = ribasim_model.level_boundary.add(Node(new_node_id, Point(74861, 382484)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(74504, 382443)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(ribasim_model.basin[133], tabulated_rating_curve)
ribasim_model.edge.add(tabulated_rating_curve, level_boundary)

In [10]:
#add the meta_node_id for the newly created TRC
ribasim_model.tabulated_rating_curve.node.df['meta_node_id'] = (
    ribasim_model.tabulated_rating_curve.node.df['meta_node_id']
    .fillna(pd.Series(ribasim_model.tabulated_rating_curve.node.df.index, 
                      index=ribasim_model.tabulated_rating_curve.node.df.index)))

In [11]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Add metadata

In [12]:
# post_processed_data = gpd.read_file(r"../../../../../Data_postprocessed/Waterschappen/Scheldestromen/Scheldestromen.gpkg")
# crossings = gpd.read_file(r"../../../../../Data_crossings/Scheldestromen/scheldestromen_crossings_v02.gpkg")
# crossings = {}
# gpkg_path = r"../../../../../Data_crossings/Scheldestromen/scheldestromen_crossings_v02.gpkg"
# with fiona.Env():
#     layers = fiona.listlayers(gpkg_path)
# for layer in layers:
#     crossings[layer] = gpd.read_file(gpkg_path, layer=layer)

# checks = ribasim_param.checks(ribasim_model)
# ribasim_model = ribasim_param.add_missing_meta_data(ribasim_model = ribasim_model, 
#                       checks = checks, 
#                       post_processed_data = post_processed_data, 
#                       crossings = crossings)


In [13]:
# #metadata has been added at the crossings_to_ribasim.py. However, some nodes have been added or altered due to the feedback forms. Update metadata, as the control (sturing) depends on it.

# #retrieve the same data as the crossings_to_ribasim.py
# model_characteristics = {}
# model_characteristics['path_crossings'] = "../../../../Data_crossings/Scheldestromen/scheldestromen_crossings_v02.gpkg"
# post_processed_data = r"../../../../Data_postprocessed/Waterschappen/Scheldestromen/Scheldestromen.gpkg"
# checks = gpd.read_file(ribasim_toml.replace('ribasim.toml', 'database_checks.gpkg'))
# crossings = gpd.read_file(r"../../../../../Data_crossings/Scheldestromen/scheldestromen_crossings_v02.gpkg")

# nodes_MD = pd.concat([ribasim_model.basin.node.df, 
#                    ribasim_model.tabulated_rating_curve.node.df, 
#                    ribasim_model.pump.node.df,
#                    ribasim_model.level_boundary.node.df,
#                    ribasim_model.flow_boundary.node.df,
#                    ribasim_model.terminal.node.df,
#                    ribasim_model.manning_resistance.node.df,
#                    ribasim_model.discrete_control.node.df,
#                   ]).reset_index()
# nodes_MD['type'] = nodes_MD['node_type']
# edges_MD = ribasim_model.edge.df

# #retrieve the same scripts as the crossings_to_ribasim.py
# RN = RibasimNetwork(nodes = nodes_MD, 
#                     edges = edges_MD,
#                     model_characteristics = model_characteristics)

# checks2 = RN.check(model = ribasim_model,
#                  post_processed_data = post_processed_data,
#                  crossings = crossings)
# stop
# RN.add_meta_data(model = ribasim_model,
#                  checks = checks,
#                  post_processed_data = post_processed_data,
#                  crossings = crossings)

### Basin (forcing)

In [14]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10*2),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [15]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [16]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

#Janna Schoonakker asked on 12-11-2024 to change the level of this specific level boundary
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==583, 'level'] = -2

#See email 6/12/2024 Janna Schoonakker to change the level of this specific level boundary
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==585, 'level'] = -2


### Add Outlet

In [17]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

In [18]:
# ribasim_model.pump.static.df.iloc[50::]

## Add control, based on the meta_categorie

In [19]:
ribasim_model.pump.static.df.loc[
    ribasim_model.pump.static.df[['meta_func_afvoer', 'meta_func_aanvoer', 'meta_func_circulatie']].isna().all(axis=1),
    'meta_func_afvoer'
] = 1

In [20]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [21]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [22]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [23]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


In [24]:
#overwrite some sturing
# ribasim_model.outlet.static.df.max_downstream_level = np.nan
# ribasim_model.pump.static.df.loc[ribasim_model.pump.static.df.meta_func_afvoer, 'max_downstream_level'] = np.nan

### Manning Resistance

In [25]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [26]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 10
ribasim_model.manning_resistance.static.df.manning_n = 0.01

In [27]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

In [28]:
ribasim_model.use_validation = True

In [29]:
# ribasim_model.edge.df["fid"] = ribasim_model.edge.df.index.copy()

# Set numerical settings

In [30]:
# Write model output
# ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2025, 1, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

In [31]:
# 1-229: basin
# 230-899: outlet
# 235-890: pump
# 900-1902: level_boundary
# 231-883: manning_resistance

# manning_resistance en outlet overlappen NIET
# manning_resistance en pump overlappen NIET


In [32]:
# all_indices = pd.concat([
#     ribasim_model.basin.node.df,
#     ribasim_model.outlet.node.df,
#     ribasim_model.pump.node.df,
#     ribasim_model.level_boundary.node.df,
#     ribasim_model.manning_resistance.node.df
# ]).index

# duplicated_indices = all_indices[all_indices.duplicated()]
# print(duplicated_indices)

# sorted_indices = all_indices.sort_values()
# print(sorted_indices)

## Run Model

## Iterate over tabulated rating curves

In [33]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [34]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

In [35]:
# try:
#     ribasim_param.iterate_TRC(
#         ribasim_param=ribasim_param,
#         allowed_tolerance=0.02,
#         max_iter=1,
#         expected_difference=0.1,
#         max_adjustment=0.25,
#         cmd=["ribasim", path_ribasim_toml],
#         output_dir=output_dir,
#         path_ribasim_toml=path_ribasim_toml,
#     )

# except Exception:
#     logging.error("The model was not able to run. Log file:")
#     log_file_path = os.path.join(output_dir, "ribasim.log")  # Update with the correct path to your log file
#     try:
#         with open(log_file_path) as log_file:
#             log_content = log_file.read()
#             print(log_content)
#     except Exception as log_exception:
#         logging.error(f"Could not read the log file: {log_exception}")

# Write model

In [36]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)

../../../../../Ribasim_networks/Waterschappen/Scheldestromen/modellen/Scheldestromen_parametrized/results
The model of waterboard Scheldestromen has been uploaded to the goodcloud in the directory of boezemmodel!


## Open Output

In [37]:
df_basin = pd.read_feather(os.path.join(output_dir, "basin.arrow"))
df_basin

,time,node_id,storage,level,inflow_rate,outflow_rate,storage_rate,precipitation,evaporation,drainage,infiltration,balance_error,relative_error
0,2024-01-01 00:00:00,1,1.520997e+06,-0.800000,0.059161,0.047403,1.882406e-01,0.176482,0.0,0.0,0.0,-2.775558e-17,-1.961201e-16
1,2024-01-01 00:00:00,2,2.798845e+06,0.900000,1.902204,1.515994,7.109616e-01,0.324752,0.0,0.0,0.0,-1.110223e-16,-5.932342e-17
2,2024-01-01 00:00:00,3,1.283031e+06,-0.700000,0.022065,0.038000,1.329367e-01,0.148871,0.0,0.0,0.0,0.000000e+00,0.000000e+00
3,2024-01-01 00:00:00,4,1.155093e+06,-1.500000,0.000099,0.030907,1.032186e-01,0.134026,0.0,0.0,0.0,-1.387779e-17,-1.681830e-16
4,2024-01-01 00:00:00,5,4.314780e+05,-0.650000,0.000657,0.021412,2.930969e-02,0.050065,0.0,0.0,0.0,-3.469447e-18,-9.619565e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346699,2024-12-31 23:00:00,823,5.067079e+05,-0.698410,0.000000,0.058747,-8.208832e-09,0.058747,0.0,0.0,0.0,1.031665e-14,1.756119e-13
3346700,2024-12-31 23:00:00,824,1.195943e+06,-0.897710,0.000000,0.138607,0.000000e+00,0.138607,0.0,0.0,0.0,2.986500e-14,2.154651e-13
3346701,2024-12-31 23:00:00,825,1.046062e+07,106.319167,0.000000,0.000000,2.599740e-01,0.259974,0.0,0.0,0.0,-1.004752e-13,-7.729634e-13
3346702,2024-12-31 23:00:00,826,2.378431e+05,-0.498841,0.000000,0.027581,-3.233759e-14,0.027581,0.0,0.0,0.0,-8.100033e-15,-2.936805e-13
